# RECOMMENDATION SYSTEM - CONTENT BASED

Saya menggunakan Recommendation System untuk mempelajari spesifikasi dari beberapa Airbnb yang telah memiliki reviews score rating 100 di setiap area yang paling profit. Rekomendasi akan dilakukan berdasarkan kemiripan score dan space yang disediakan oleh Airbnb tersebut. Dengan memprovide rekomendasi ini, kita akan mengetahui apa yang disukai pasar, spesifikasi apa yang dibutuhkan sebuah airbnb, dan tentu nya akan memudahkan sang pengusaha untuk mendapatkan keuntungan.

In [30]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.corpus import stopwords
import re
import random

In [31]:
airbnb = pd.read_csv('./Desktop/listings-2.csv', usecols = ['id','neighbourhood_cleansed', 'name', 'review_scores_rating'])

In [32]:
airbnb['neighbourhood_cleansed'].value_counts().head(5)

Shinjuku Ku    2770
Taito Ku       2048
Toshima Ku     1553
Sumida Ku      1339
Shibuya Ku     1052
Name: neighbourhood_cleansed, dtype: int64

In [33]:
airbnb_top_shinjuku = airbnb[(airbnb['review_scores_rating']==100)&(airbnb['neighbourhood_cleansed']=="Shinjuku Ku")]
airbnb_top_taitoku = airbnb[(airbnb['review_scores_rating']==100)&(airbnb['neighbourhood_cleansed']=="Taito Ku")]
airbnb_top_toshimaku = airbnb[(airbnb['review_scores_rating']==100)&(airbnb['neighbourhood_cleansed']=="Toshima Ku")]
airbnb_top_sumidaku = airbnb[(airbnb['review_scores_rating']==100)&(airbnb['neighbourhood_cleansed']=="Sumida Ku")]
airbnb_top_shibuya = airbnb[(airbnb['review_scores_rating']==100)&(airbnb['neighbourhood_cleansed']=="Shibuya Ku")]

In [34]:
airbnb_top_shinjuku.head(1)

,id,name,neighbourhood_cleansed,review_scores_rating
374,8398953,EXPERIENCE JAPAN with my family!!,Shinjuku Ku,100.0


In [35]:
airbnb_top_taitoku.head(1)

,id,name,neighbourhood_cleansed,review_scores_rating
457,9467882,YATSUDE･Japanese room /shared bathroom(1F),Taito Ku,100.0


In [36]:
airbnb_top_toshimaku.head(1)

,id,name,neighbourhood_cleansed,review_scores_rating
1184,15906951,"HOUSE, 2 pocket Wifi, 4 min walk to Ikebukuro ...",Toshima Ku,100.0


In [37]:
airbnb_top_sumidaku.head(1)

,id,name,neighbourhood_cleansed,review_scores_rating
1982,20607843,"5min AKIHABARA, 10min ASAKUSA COZY 4ppl PRIVAT...",Sumida Ku,100.0


In [38]:
airbnb_top_shibuya.head(1)

,id,name,neighbourhood_cleansed,review_scores_rating
576,10582789,♥ようこそ Hygge cat themed home Yoyogi park Shibuya♥,Shibuya Ku,100.0


In [44]:
airbnb = pd.read_csv('./Desktop/listings-2.csv', usecols = ['id', 'name', 'space','review_scores_rating'])
airbnb.head()

,id,name,space,review_scores_rating
0,35303,"La Casa Gaienmae C Harajuku, Omotesando is nearby",This apartment is 3 bedroom flat shared with t...,94.0
1,197677,Oshiage Holiday Apartment,"We are happy to welcome you to our apartment, ...",95.0
2,289597,Private apt in central Tokyo #203,::::::::::::::::::::::::::::::::::::::::::::::...,95.0
3,370759,"Cozy flat #203, local area YET 10 mins to shib...","Cozy and Relaxing, at home feeling in a reside...",95.0
4,700253,Private apt in central Tokyo #201,::::::::::::::::::::::::::::::::::::::::::::::...,96.0
5,872235,RELAX AT HOME:Shimo 5'/Shibuya 10',Nice LITTLE house with atmosphere in Tokyo. Be...,96.0
6,899003,"Classy room @Shinjuku, Takadanoba",Walkable from Shinjuku station even if you mis...,90.0
7,905944,4F - Near Shinjuku & Shibuya w/Free WiFi,Welcome and thank you for reading the details ...,95.0
8,1016831,WOMAN ONLY LICENSED ! Cosy & Cat behnd Shibuya,We offer quite and peaceful private 6 tatami r...,99.0
9,1096292,Home stay with a Japanese language teacher fam...,★*:.★*:. Must Read 2 .:*★.:*★ *:.Your private...,98.0


In [45]:
def print_description(index):
    example = airbnb[airbnb.index == index][['id', 'name', 'space','review_scores_rating']].values[0]
    if len(example) > 0:
        print(example[0])
        print('Nama:', example[1])
        print('Space:\n',example[2])  
        print('Rating:\n',example[3])

In [46]:
print_description(1)

197677
Nama: Oshiage Holiday Apartment
Space:
 We are happy to welcome you to our apartment, located in the heart of Tokyo downtown. This is an authentic Japanese apartment with Tatami mattress room and sleeping on Japanese Futon, like Ryokan style. Fully equipped and convienient kitchen will give you oportunity to feel like at home. Automatic bath tub. Separate toilet with heating seat and washlet. Direct acces from both Narita and Haneda airports. Easy access to most of Tokyo attractions. 10min walk from Oshiage Station, 7min walk from Tobu Hikifune Station, 8min walk from Heisei Hikifune Station. Free internet access. Air conditioning, 2 semi-double futon bed (for 2 person each), LCD 32 inch TV, full kitchen, microwave, toster, electric pot, refrigerator, coffee maker, iron, hair dryer, washing machine, bathroom with a bath tub and shower, gas grill. Cooking utensils and linens provided. Our apartment is located in the center of Tokyo downtown, where you can still taste real traditi

In [47]:
airbnb['content'] = airbnb[['name', 'space','review_scores_rating']].astype(str).apply(lambda x: ' // '.join(x), axis = 1)

In [48]:
tf = TfidfVectorizer(analyzer = 'word', ngram_range = (1, 2), min_df = 0, stop_words = 'english')
tfidf_matrix = tf.fit_transform(airbnb['content'])

In [49]:
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

In [50]:
results = {}
for idx, row in airbnb.iterrows():
    similar_indices = cosine_similarities[idx].argsort()[:-100:-1]
    similar_items = [(cosine_similarities[idx][i], airbnb['id'][i]) for i in similar_indices]
    results[row['id']] = similar_items[1:]

In [51]:
def item(neighbourhood_cleansed):
    name   = airbnb.loc[airbnb['id'] == neighbourhood_cleansed]['content'].tolist()[0].split(' // ')[0]
    desc   = ' \nSpace: ' + airbnb.loc[airbnb['id'] == neighbourhood_cleansed]['content'].tolist()[0].split(' // ')[1][0:165] + '...'
    prediction = name  + desc
    return prediction

def recommend(kota, num):
    print('Rekomendasi ' + str(num) + ' listing yang mirip dengan ' + item(kota))
    print('---')
    recs = results[kota][:num]
    for rec in recs:
        print('\nNama : ' + item(rec[1]))

In [52]:
airbnb_top_shinjuku.head(1)

,id,name,neighbourhood_cleansed,review_scores_rating
374,8398953,EXPERIENCE JAPAN with my family!!,Shinjuku Ku,100.0


In [53]:
airbnb_top_taitoku.head(1)

,id,name,neighbourhood_cleansed,review_scores_rating
457,9467882,YATSUDE･Japanese room /shared bathroom(1F),Taito Ku,100.0


In [54]:
airbnb_top_toshimaku.head(1)

,id,name,neighbourhood_cleansed,review_scores_rating
1184,15906951,"HOUSE, 2 pocket Wifi, 4 min walk to Ikebukuro ...",Toshima Ku,100.0


In [55]:
airbnb_top_sumidaku.head(1)

,id,name,neighbourhood_cleansed,review_scores_rating
1982,20607843,"5min AKIHABARA, 10min ASAKUSA COZY 4ppl PRIVAT...",Sumida Ku,100.0


In [56]:
airbnb_top_shibuya.head(1)

,id,name,neighbourhood_cleansed,review_scores_rating
576,10582789,♥ようこそ Hygge cat themed home Yoyogi park Shibuya♥,Shibuya Ku,100.0


In [58]:
ID = int(input("Masukkan ID: "))
JUMLAH = int(input("Masukkan Jumlah Pencarian: \n"))
recommend(kota = ID ,num = JUMLAH)

Masukkan ID:  15906951
Masukkan Jumlah Pencarian: 
 5


Rekomendasi 5 listing yang mirip dengan HOUSE, 2 pocket Wifi, 4 min walk to Ikebukuro Stat 
Space: This home is located in a quiet residential area, and can accommodate up to 8 guests. It is in the perfect location for exploring Tokyo, as it is near Ikebukuro stat...
---

Nama : House, 2 mobile wifi, 4 mins walk to JR Station 
Space: This house is located in a quiet residential area, and can house up to 5 guests. There are two independent bedrooms, a living room, kitchen, separate toilet and bath...

Nama : GR202 Akasaka, Roppongi/8mins to Shinjuku, Shibuya 
Space: Modern, brand new 1BR apartment space with Japanese styled interior. Can  accommodate up to guests. Perfect for couples and single travelers.   ★Amenities★ In additi...

Nama : GR201 Nr Roppongi/2pax/Direct to Shinjuku, Shibuya 
Space: Modern, brand new 1BR apartment space with Japanese styled interior. Can accommodate up to 2 people - perfect for couples or single travelers.  ★Amenities★ In additi...

Nama : Luxury 3LDK Hom